# Generate annotated data in Conll format

In [1]:
from collections import OrderedDict
import os
# docs used in annotations
txt_dir = '/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/PDF_PAGES_INTO_TXTFILES_WITH_LAYOUT/'
def get_docnames():
    dir_list= os.listdir(txt_dir)
    #doc_2_annotation = [{doc_2_annotation[x] = []} 
    doc_2_annotation = OrderedDict()
    for x in dir_list:
        doc_2_annotation[x] = []
    return doc_2_annotation

doc_2_annotation = get_docnames()

In [2]:
#bean class to hold annotation attributes
class Annotation():
    def __init__(self, doc_name, begin, end, entity_text, entity_type, add_to_conll=False):
        self.doc_name = doc_name
        self.begin = begin
        self.end = end
        self.entity_text = entity_text
        self.entity_type = entity_type
        self.add_to_conll = add_to_conll
    def __str__(self):
        return self.doc_name + ' ' + str(self.begin) + ' ' + str(self.end) + ' ' + self.entity_text + ' ' + self.entity_type
    def __eq__(self,other):
        return self.begin == other.begin and self.end == other.end \
                and self.entity_text == other.entity_text and self.entity_type == other.entity_type

In [3]:
def read_annotations(annotation_file_name, doc_2_annotation):
    annotation_list = []
    with open(annotation_file_name) as reader:
        for line in reader:
            res = line.split('\t')
            #print(res)
            annotation = Annotation(res[4]+".txt", int(res[2]), int(res[3]), [1], "CHEMICAL")
            docname = res[4]+".txt"
            #print(docname)
            if docname not in doc_2_annotation.keys():
                continue
            doc_2_annotation[docname].append(annotation)
            annotation_list.append(annotation)
    return doc_2_annotation, annotation_list

In [7]:
def get_type(annotation_list, begin, end):
    if end - begin == 1:
        return "O"
    for antn in annotation_list:
        antn_type = antn.entity_type
        antn_begin = antn.begin
        antn_end = antn.end
        if begin <= antn_begin and antn_begin < end:
            antn.add_to_conll = True
            return antn_type
            #return 'B-'+antn_type
        elif (antn_begin < begin and begin < antn_end) or (antn_begin < end  and end <= antn_end):
            #antn.add_to_conll = True
            return antn_type
    return "O"

In [8]:
def remove_overlap_anotn(annotation_list):
    #clean_annoation_list = []
    flag = False
    for x in annotation_list:
        for y in annotation_list:
            if x != y:
                #check if y is a part of x
                if x.begin <= y.begin and y.end < x.end:
                    print("Overlap: ")
                    print(str(y) + "is a part of \n" + str(x))
                    annotation_list.remove(y)
    return annotation_list  

def check_annotation_not_added_in_Conll(annotation_list):
    for x in annotation_list:
        if not x.add_to_conll:
            print(str(x))

In [9]:
import spacy
import re
#import nltk
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
# from spacy.pipeline import Sentencizer
# sentencizer = Sentencizer()
config = {"punct_chars": None}
nlp.add_pipe("sentencizer", config=config)

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [ ]:
#Generate TSV version of annotation
import re
list_str = ""
for line in open("list.txt").readlines():
    line = line.strip().lower()
    if len(line) == 0:
        continue
    list_str+=re.escape(line)+"|"+re.escape(line+"s")+"|"
    
list_str = list_str[:-1]

regex = "\\b("+list_str+")\\b"
#print(regex)

def custom_tokenizer(token):
    actual_token = token
    #print(actual_token)
    token = token.lower()
    final_tokens = []
    match_beg_end_list = []
    for i, match in enumerate(re.finditer(regex, token)):
        #print(i, match.group())
        beg = match.span()[0]
        end = match.span()[1]
        match_beg_end_list.append((beg, end))
    start = 0
    end = len(actual_token)
    for i, x in enumerate(match_beg_end_list):
        if start != x[0]:
            final_tokens.append(actual_token[start:x[0]])
        final_tokens.append(actual_token[x[0]:x[1]])
        start = x[1]
    if len(actual_token[end:]) > 0:
        final_tokens.append(actual_token[end:])
    if len(final_tokens) == 0:
        final_tokens.append(actual_token)
    return final_tokens

print(custom_tokenizer("C.F.R."))

In [ ]:
annotation_file_name = "annotation_validated.tsv"
doc_2_annotation, annotation_list = read_annotations(annotation_file_name, doc_2_annotation)
# output connll file
out_conll_path = "annotation.conll"
writer = open(out_conll_path, 'w')
flag = False
annotation_counter = 0
entity_mismatch = 0
entity_split_by_newline_counter = 0
NON_ENTITY_DOC  = 0
print(doc_2_annotation.keys())
for doc_name in doc_2_annotation.keys():
    annotation_list = doc_2_annotation[doc_name]
    if len(annotation_list) == 0:
        NON_ENTITY_DOC +=1
        continue
    annotation_list = remove_overlap_anotn(annotation_list)
    annotation_counter = annotation_counter + len(annotation_list)
    doc = open(txt_dir+doc_name).read() 
    #print(doc_name, doc)
    entity_idx = 0
    prev_tag = ''
    offset = 0
    for paragraph in re.split("[\n\r]", doc):
        sentences = nlp(paragraph)
        for sentence in sentences.sents:
            if sentence.text:
                tokens_ = nlp(sentence.text)
                for tok in tokens_:
                    tokens = custom_tokenizer(tok.text)
                    for token in tokens:
                        offset = doc.find(token, offset)
                        begin = offset
                        end = offset + len(token)
                        offset += len(token)
                        if begin == -1:
                            print("Begin starts with -1")
                            print(sentence.text)
                            print(str(token) + '\t' + str(begin) + '\t' + str(end)  + '\t' + doc_name+'\t'+entity_type+'\n')
                            print("#################################################################")
                        entity_type = get_type(annotation_list, begin, end)
                        writer.write(str(token) + '\t' + str(begin) + '\t' + str(end)  + '\t' + doc_name+'\t'+entity_type+'\n')
                writer.write('\n')
writer.close()
print('# of annotations in conll: ', annotation_counter)
print('Entity Mismatch: ', entity_mismatch)
print('Entity Split by new line: ', entity_split_by_newline_counter)
print("Non enitity docs: ", NON_ENTITY_DOC)

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
#!gdown --id 13758oOJgPmqEQITQpdT2cRaXNHIBCQoU 
!gdown --id 13758oOJgPmqEQITQpdT2cRaXNHIBCQoU
!unzip 5-fold.zip

#https://drive.google.com/file/d/13758oOJgPmqEQITQpdT2cRaXNHIBCQoU/view?usp=sharing

In [ ]:
!head -n 5 "5-fold/testData_1".txt

In [ ]:
import json
def generate_datasets():
    for i in range(1, 6):
        print(i)
        writer = open(f'/content/train{i}.json', 'w')
        words = []
        ner = []
        with open(f'/content/5-fold/trainData_{i}.txt') as reader:
            for line in reader:
                items = line.strip().split('\t')
                #print(items)
                if len(items) < 3 and len(words) > 1:
                    writer.write(json.dumps({"words":words, "ner_tags": ner}))
                    writer.write('\n')
                    words = []
                    ner = []
                    continue
                if len(items) > 3:
                    words.append(items[0])
                    ner.append(items[-1])

        writer = open(f'/content/test{i}.json', 'w')
        words = []
        ner = []
        with open(f'/content/5-fold/testData_{i}.txt') as reader:
            for line in reader:
                items = line.strip().split('\t')
                if len(items) < 3 and len(words) > 1:
                    writer.write(json.dumps({"words":words, "ner_tags": ner}))
                    writer.write('\n')
                    words = []
                    ner = []
                    continue
                if len(items) > 3:
                    words.append(items[0])
                    ner.append(items[-1])

generate_datasets()

##Model Training

In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r /content/transformers/examples/pytorch/token-classification/requirements.txt
!pip install datasets

In [ ]:
#FOLD = 1
import os
os.makedirs(f"/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/bert-base-cased/checkpoint1", exist_ok=True)
%cd /content/transformers/examples/pytorch/token-classification
!python run_ner.py \
  --model_name_or_path "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/WaterBERT/water-bert-cased" \
  --train_file /content/train1.json \
  --validation_file  /content/test1.json \
  --test_file  /content/test1.json \
  --output_dir "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint1" \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --do_predict \
  --num_train_epochs 3\
  --evaluation_strategy epoch\
  --overwrite_output_dir True\
  --save_steps=10000

In [ ]:
#FOLD = 2
import os
os.makedirs(f"/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint2", exist_ok=True)
%cd /content/transformers/examples/pytorch/token-classification
!python run_ner.py \
  --model_name_or_path "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/WaterBERT/water-bert-cased" \
  --train_file /content/train2.json \
  --validation_file  /content/test2.json \
  --test_file  /content/test2.json \
  --output_dir "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint2" \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --do_predict \
  --num_train_epochs 3\
  --evaluation_strategy epoch\
  --overwrite_output_dir True\
  --save_steps=10000

In [ ]:
#FOLD = 3
import os
os.makedirs(f"/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint3", exist_ok=True)
%cd /content/transformers/examples/pytorch/token-classification
!python run_ner.py \
  --model_name_or_path "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/WaterBERT/water-bert-cased" \
  --train_file /content/train3.json \
  --validation_file  /content/test3.json \
  --test_file  /content/test3.json \
  --output_dir "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint3" \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --do_predict \
  --num_train_epochs 3\
  --evaluation_strategy epoch\
  --overwrite_output_dir True\
  --save_steps=10000

In [ ]:
#FOLD = 4
import os
os.makedirs(f"/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint4", exist_ok=True)
%cd /content/transformers/examples/pytorch/token-classification
!python run_ner.py \
  --model_name_or_path "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/WaterBERT/water-bert-cased" \
  --train_file /content/train4.json \
  --validation_file  /content/test4.json \
  --test_file  /content/test4.json \
  --output_dir "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint4" \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --do_predict \
  --num_train_epochs 3\
  --evaluation_strategy epoch\
  --overwrite_output_dir True\
  --save_steps=10000

In [ ]:
#FOLD = 5
import os
os.makedirs(f"/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint5", exist_ok=True)
%cd /content/transformers/examples/pytorch/token-classification
!python run_ner.py \
  --model_name_or_path "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/WaterBERT/water-bert-cased" \
  --train_file /content/train5.json \
  --validation_file  /content/test5.json \
  --test_file  /content/test5.json \
  --output_dir "/content/drive/MyDrive/CSCE Fall2022-Raxit/Course Project/Experiment_Validated_Annotation/Mississippi/Results/water-bert-cased/checkpoint5" \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 16 \
  --do_train \
  --do_eval \
  --do_predict \
  --num_train_epochs 3\
  --evaluation_strategy epoch\
  --overwrite_output_dir True\
  --save_steps=10000

In [ ]:
#scibert-scivocab-uncased
#chemical-bert-uncased
#dmis-lab/biobert-base-cased-v1.2